# Create CFI Awards

Creates CFI (Canada Foundation for Innovation) awards from open data. ~14K projects.

**Prerequisites:**
- Run `scripts/local/cfi_to_s3.py` to download and upload the data first.

**Data source:** https://www.innovation.ca/projects-results/funded-projects-dashboard  
**S3 location:** `s3a://openalex-ingest/awards/cfi/cfi_projects.parquet`

**CFI funder:**
- funder_id: 4320320995
- display_name: "Canada Foundation for Innovation"
- ROR: https://ror.org/000az4664
- DOI: 10.13039/501100001805

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.cfi_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/cfi/cfi_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~14K)
SELECT COUNT(*) as total_projects FROM openalex.awards.cfi_raw;

In [ ]:
%sql
-- Inspect the schema
DESCRIBE openalex.awards.cfi_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.cfi_raw LIMIT 5;

## Step 2: Create CFI Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.cfi_awards
USING delta
AS
WITH
-- Get CFI funder from OpenAlex by explicit funder_id
cfi_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320320995  -- Canada Foundation for Innovation
),

awards_transformed AS (
    SELECT
        -- Generate unique ID: funder_id:row_number (CFI doesn't have native award IDs)
        CONCAT(f.funder_id, ':', LOWER(REGEXP_REPLACE(CONCAT(g.decision_year, '_', g.lead_institution, '_', SUBSTRING(g.project_title, 1, 50)), '[^a-zA-Z0-9]', '_'))) as id,

        -- Display name = project title
        g.project_title as display_name,

        -- No abstract in CFI data
        CAST(NULL AS STRING) as description,

        -- Funder info
        f.funder_id,
        -- Create a pseudo award ID from year + institution + title hash
        CONCAT('CFI-', g.decision_year, '-', ABS(HASH(CONCAT(g.lead_institution, g.project_title))) % 1000000) as funder_award_id,

        -- Amount
        TRY_CAST(g.cfi_contribution AS DOUBLE) as amount,
        'CAD' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - CFI is infrastructure funding
        'infrastructure' as funding_type,

        -- Funder scheme = fund type
        g.fund_type as funder_scheme,

        -- Provenance
        'cfi_opendata' as provenance,

        -- Dates
        TRY_TO_DATE(g.decision_date) as start_date,
        CAST(NULL AS DATE) as end_date,
        TRY_CAST(g.decision_year AS INT) as start_year,
        CAST(NULL AS INT) as end_year,

        -- Lead investigator - CFI has team_leaders which may contain multiple names
        -- Take the first name before any pipe separator
        CASE
            WHEN g.team_leaders IS NOT NULL AND TRIM(g.team_leaders) != '' THEN
                struct(
                    CAST(NULL AS STRING) as given_name,
                    TRIM(SPLIT(g.team_leaders, '\\|')[0]) as family_name,  -- Store full name in family_name
                    CAST(NULL AS STRING) as orcid,
                    TRY_TO_DATE(g.decision_date) as role_start,
                    struct(
                        TRIM(g.lead_institution) as name,
                        'Canada' as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not structured in CFI data)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL - CFI dashboard
        'https://www.innovation.ca/projects-results/funded-projects-dashboard' as landing_page_url,

        -- No DOI for CFI projects
        CAST(NULL AS STRING) as doi,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.cfi_raw g
    CROSS JOIN cfi_funder f
    WHERE g.project_title IS NOT NULL
      AND TRIM(g.project_title) != ''
)

SELECT * FROM awards_transformed;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~14K)
SELECT COUNT(*) as total_cfi_awards FROM openalex.awards.cfi_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    currency,
    start_date,
    lead_investigator
FROM openalex.awards.cfi_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be CFI)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.cfi_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution (top 10 fund types)
SELECT funder_scheme, COUNT(*) as cnt
FROM openalex.awards.cfi_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 10;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_abstract,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(lead_investigator) as has_pi,
    ROUND(COUNT(display_name) * 100.0 / COUNT(*), 1) as pct_title,
    ROUND(COUNT(amount) * 100.0 / COUNT(*), 1) as pct_amount,
    ROUND(COUNT(start_date) * 100.0 / COUNT(*), 1) as pct_dates
FROM openalex.awards.cfi_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt
FROM openalex.awards.cfi_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 15;

In [ ]:
%sql
-- Check lead institutions (top 15)
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as cnt
FROM openalex.awards.cfi_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY cnt DESC
LIMIT 15;

In [ ]:
%sql
-- Check total funding amount
SELECT 
    ROUND(SUM(amount) / 1000000000, 2) as total_funding_billions_cad,
    ROUND(AVG(amount), 0) as avg_award_cad,
    ROUND(PERCENTILE_APPROX(amount, 0.5), 0) as median_award_cad
FROM openalex.awards.cfi_awards
WHERE amount IS NOT NULL;